# Sesión 06 - Manejo de Modelos de Clasificación para entornos operativos

*Objetivo*

- Aplicar algoritmos de clasificación para identificar eventos operacionales críticos en la industria petrolera.
- Evaluar el rendimiento de diferentes clasificadores.
- Interpretar los resultados y la matriz de confusión.
- Analizar la eficiencia en pozos productores.

In [ ]:
!python --version


## 0. Carga de Librerías

In [ ]:
# Instalación de dependencias del nuevo codespace
!pip install -f ../requirements.txt

In [ ]:
# Importación de Librerías

## 1. Carga de Datos

In [ ]:
# Carga desde la carpeta data/raw/

## 2. EDA (Medidas de Tendencia Central, Análisis de Nulos)

In [ ]:
# Métodos info(), describe()

## 3. Transformaciones (Imputación, Encoding)

## 4. Modelamiento

In [ ]:
# Carga de librerías para modelos

## 5. Evaluación